In [ ]:
from tensorflow import keras
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow import nn
import collections
from rdkit.Chem import Descriptors
from rdkit import Chem
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, accuracy_score, multilabel_confusion_matrix

In [ ]:
data = pd.read_csv('solvent_dataset.csv', sep="\t",low_memory=False)

In [ ]:
#params
#for quick prototyping epoch count is set to 20
epoch_count = 20

In [ ]:
def get_best_s():
    zest = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
    zesp = 0.2
    zel = []
    maxpr = 0
    best_nr = 0
    bad_count = 0
    for zzyz in range(0,100):

        try:
            preds1 = pred_model.predict(X_test2,verbose=0,batch_size=512)
            #print(preds1)
            #print(y_test2)
            preds1[preds1>=zesp] = 1
            preds1[preds1<1-zesp] = 0
            klkl = accuracy_score(y_test2_2, preds1)

            if klkl > maxpr:
                maxpr = klkl
                best_nr = zesp
            #print (klkl)
        except:
            bad_count +=1
        if bad_count >4:
            break
        zesp+=0.005
        zel.append(klkl)
    #print(best_nr)
    #plt.plot(zel)
    print(maxpr)
    #print(np.array(zel).max())
    #plt.plot(zel)
    return best_nr

In [ ]:
#Generate morgan FP, may take a while
bins_len= 512
smiles_train = data["RP"]
def vectorize(smiles):
    x_one = np.zeros((len(data), 8, bins_len ),dtype=np.int8)
    #x_one = []
    for ca,smil in enumerate(smiles):
        #x_one_ = []
        qq = list(smil.split("."))
        for fa in range(0,8):
            try:
                x_one[ca][fa]=np.fromiter((map(int, AllChem.GetMorganFingerprintAsBitVect(AllChem.MolFromSmiles(qq[fa]),3,nBits=bins_len).ToBitString())),int)
            except:
                pass
            #x_one_.append([0 for x in range(0,bins_len)])
            #print(smil)
        
    return x_one
X_train = vectorize(smiles_train.values)

In [ ]:
#saving a numpy array is recommended, to avoid generating morgan FP every time
#np.save('X_train_rat.npy', X_train)
# npy file can be loaded with:
#X_train = np.load("X_train_rat.npy")

In [ ]:
X_train_flat = X_train.reshape((len(X_train), np.prod(X_train.shape[1:])))

In [ ]:
charset_t = collections.Counter((" ".join(list(data.crude))).split(' '))
print(charset_t)
charset_t=set(charset_t)
char_to_int_t = dict((c,i) for i,c in enumerate(charset_t))
int_to_char_t = dict((i,c) for i,c in enumerate(charset_t))
print(char_to_int_t)
embed_t =  len(charset_t)
print(embed_t)
one_hot_y = np.zeros((len(data), (len(charset_t)) ),dtype=np.int8)
len(one_hot_y[0])
for i,nam in enumerate(data['crude']):
    list_short = list(nam.split(" "))
    first = True
    for z, name in enumerate(list_short):
        if first == True:
            one_hot_y[i][char_to_int_t[name]] = 1
            #first = False
        #elif first !=True:
        #    one_hot_y[i][char_to_int_t[name]+11] = 1
            
Y_train = np.array(one_hot_y)

In [ ]:
X_train2, X_test2, y_train2_2,y_test2_2  = train_test_split(X_train,X_train_flat, test_size=0.1,shuffle=True)

In [ ]:
######## LSTM ~~~~~~~~~~~~~~
encoding_dim = 512  

# This is our input image
input_img = keras.Input(shape=(8,512,))
# "encoded" is the encoded representation of the input
encoded = layers.LSTM(encoding_dim)(input_img)
# "decoded" is the lossy reconstruction of the input

decoded = layers.Dense(4096, activation='sigmoid')(encoded)

autoencoder = keras.Model(input_img, decoded)
#encoder = keras.Model(input_img, encoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
encoder = keras.Model(input_img, encoded)
encoder.compile(optimizer='adam', loss='binary_crossentropy')
encoder.summary()

In [ ]:
encoder = keras.Model(input_img, encoded)
encoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
history=autoencoder.fit(X_train2, y_train2_2,epochs=20,batch_size=512,shuffle=True)

In [ ]:
input_img = keras.Input(shape=(8,512,))

#input_to_model = encoder
#x = layers.Dense(32,activation='relu')(input_to_model)
'''
enc2 = encoder.layers[-8](input_img)
enc2 = encoder.layers[-7](enc2)
enc2 = encoder.layers[-6](enc2)
enc2 = encoder.layers[-5](enc2)

enc2 = encoder.layers[-4](enc2)
enc2 = encoder.layers[-3](enc2)


enc2 = encoder.layers[-2](enc2)
'''
enc2 = autoencoder.layers[1](input_img)

#ls = layers.Dense(64, activation="relu")(enc2)
ls = layers.Dense(256, activation="relu")(enc2)
ls = layers.Dense(128, activation="relu")(ls)
p = layers.Dense(10, activation="sigmoid",dtype='float32',name="classification")(ls)


pred_model = keras.Model(input_img, p)
pred_model.compile(loss='binary_crossentropy', optimizer="adam", metrics='binary_accuracy')
pred_model.layers[1].trainable=False
pred_model.summary()

In [ ]:
X_train2, X_test2, y_train2_2,y_test2_2  = train_test_split(X_train,Y_train, test_size=0.1,shuffle=True)

In [ ]:
history=pred_model.fit(X_train2, y_train2_2,epochs=epoch_count,batch_size=256,shuffle=True,validation_split=0.1)

In [ ]:
preds1 = pred_model.predict(X_test2,verbose=1,batch_size=512)
#print(preds1)
#print(y_test2)
ac_sc = get_best_s()
preds1[preds1>=ac_sc] = 1
preds1[preds1<1-ac_sc] = 0
print (accuracy_score(y_test2_2, preds1))
print (classification_report(y_test2_2, preds1,target_names = char_to_int_t.keys(), digits = 3))
print (multilabel_confusion_matrix(y_test2_2, preds1))

In [ ]:
X_train2, X_test2, y_train2_2,y_test2_2  = train_test_split(X_train_flat,X_train_flat, test_size=0.1,shuffle=True)

In [ ]:
######## Dense ~~~~~~~~~~~~~~
encoding_dim = 512  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# This is our input image
input_img = keras.Input(shape=(4096,))
# "encoded" is the encoded representation of the input
encoded = layers.Dense(encoding_dim,activation='relu')(input_img)
# "decoded" is the lossy reconstruction of the input

decoded = layers.Dense(4096, activation='sigmoid')(encoded)
#resh = layers.Reshape((8,512))(encoded)
# This model maps an input to its reconstruction
autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.summary()

In [ ]:
history=autoencoder.fit(X_train2, y_train2_2,epochs=20,batch_size=512,shuffle=True)

In [ ]:
input_img = keras.Input(shape=(4096,))


enc2 = autoencoder.layers[1](input_img)

#ls = layers.Dense(64, activation="relu")(enc2)
ls = layers.Dense(256, activation="relu")(enc2)
ls = layers.Dense(128, activation="relu")(ls)
p = layers.Dense(10, activation="sigmoid",dtype='float32',name="classification")(ls)


pred_model = keras.Model(input_img, p)
pred_model.compile(loss='binary_crossentropy', optimizer="adam", metrics='binary_accuracy')
pred_model.layers[1].trainable=False
pred_model.summary()

In [ ]:
X_train2, X_test2, y_train2_2,y_test2_2  = train_test_split(X_train,Y_train, test_size=0.1,shuffle=True)

In [ ]:
history=pred_model.fit(X_train2, y_train2_2,epochs=epoch_count,batch_size=256,shuffle=True,validation_split=0.1)

In [ ]:
preds1 = pred_model.predict(X_test2,verbose=1,batch_size=512)

ac_sc = get_best_s()
preds1[preds1>=ac_sc] = 1
preds1[preds1<1-ac_sc] = 0
print (accuracy_score(y_test2_2, preds1))
print (classification_report(y_test2_2, preds1))
print (multilabel_confusion_matrix(y_test2_2, preds1))

In [ ]:
X_train2, X_test2, y_train2_2,y_test2_2  = train_test_split(X_train,X_train, test_size=0.1,shuffle=True)

In [ ]:
encoding_dim = 512  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# This is our input image
input_img = keras.Input(shape=(8,512,))
# "encoded" is the encoded representation of the input
a = layers.Permute((2,1))(input_img)
x = layers.Conv1D(256,12,activation="relu")(a)
x1 = layers.MaxPooling1D(2)(x)
x2 = layers.Conv1D(128,10, activation='relu')(x1)
x3 = layers.MaxPooling1D(2)(x2)
x3 = layers.Conv1D(64,10, activation='relu')(x3)
x4 = layers.AveragePooling1D()(x3)
flat = layers.Flatten()(x4)
encoded = layers.Dense(encoding_dim)(flat)
d2 = layers.Reshape((128,4))(encoded)
d3 = layers.Conv1D(256,1,strides=1, activation='relu')(d2)
d4 = layers.UpSampling1D(2)(d3)
d5 = layers.Conv1D(256,1,strides=1, activation='relu')(d4)
d6 = layers.UpSampling1D(2)(d5)
decoded = layers.Conv1D(8,1,strides=1, activation='sigmoid')(d6)
decoded = layers.Permute((2,1))(decoded)
autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.summary()

In [ ]:
history=autoencoder.fit(X_train2, y_train2_2,epochs=20,batch_size=512,shuffle=True)

In [ ]:
input_img = keras.Input(shape=(8,512,))

#input_to_model = encoder
#x = layers.Dense(32,activation='relu')(input_to_model)

enc2 = autoencoder.layers[1](input_img)
enc2 = autoencoder.layers[2](enc2)
enc2 = autoencoder.layers[3](enc2)
enc2 = autoencoder.layers[4](enc2)

enc2 = autoencoder.layers[5](enc2)
enc2 = autoencoder.layers[6](enc2)


enc2 = autoencoder.layers[7](enc2)
enc2 = autoencoder.layers[8](enc2)
enc2 = autoencoder.layers[9](enc2)
#ls = layers.Dense(64, activation="relu")(enc2)
ls = layers.Dense(128, activation="relu")(enc2)
ls = layers.Dense(64, activation="relu")(ls)
p = layers.Dense(10, activation="sigmoid",dtype='float32',name="classification")(ls)


pred_model = keras.Model(input_img, p)
pred_model.compile(loss='binary_crossentropy', optimizer="adam", metrics='binary_accuracy')
pred_model.layers[1].trainable=False
pred_model.layers[2].trainable=False
pred_model.layers[3].trainable=False
pred_model.layers[4].trainable=False
pred_model.layers[5].trainable=False
pred_model.layers[6].trainable=False
pred_model.layers[7].trainable=False
pred_model.layers[8].trainable=False
pred_model.layers[9].trainable=False
pred_model.summary()

In [ ]:
X_train2, X_test2, y_train2_2,y_test2_2  = train_test_split(X_train,Y_train, test_size=0.1,shuffle=True)

In [ ]:
history=pred_model.fit(X_train2, y_train2_2,epochs=epoch_count,batch_size=256,shuffle=True,validation_split=0.1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
preds1 = pred_model.predict(X_test2,verbose=1,batch_size=512)
#print(preds1)
#print(y_test2)
ac_sc = get_best_s()
preds1[preds1>=ac_sc] = 1
preds1[preds1<1-ac_sc] = 0
print (accuracy_score(y_test2_2, preds1))
print (classification_report(y_test2_2, preds1))
print (multilabel_confusion_matrix(y_test2_2, preds1))